In [ ]:
# asennetaan transformers ja dataset

!pip install --upgrade --force-reinstall tqdm
!pip --quiet install transformers
!pip --quiet install datasets
!pip --quiet install sklearn

     |████████████████████████████████| 76 kB 3.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
     |████████████████████████████████| 2.6 MB 9.0 MB/s 
     |████████████████████████████████| 636 kB 68.0 MB/s 
     |████████████████████████████████| 895 kB 54.7 MB/s 
     |████████████████████████████████| 3.3 MB 59.2 MB/s 
     |████████████████████████████████| 264 kB 8.1 MB/s 
     |████████████████████████████████| 118 kB 69.7 MB/s 
     |████████████████████████████████| 243 kB 57.1 MB/s 


In [ ]:
# tuodaan kirjastot

from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorForTokenClassification
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

In [ ]:
# haetaan aineisto
!wget -nc -O PER-ORG-relations-combined-group-2.tsv http://dl.turkunlp.org/TKO_8964_2021/PER-ORG-relations-combined-group-2.tsv
data = pd.read_csv(("PER-ORG-relations-combined-group-2.tsv"), header=0, names=['label','text'], sep='\t')

print(data)

--2021-08-15 18:54:43--  http://dl.turkunlp.org/TKO_8964_2021/PER-ORG-relations-combined-group-2.tsv
Resolving dl.turkunlp.org (dl.turkunlp.org)... 195.148.30.23
Connecting to dl.turkunlp.org (dl.turkunlp.org)|195.148.30.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 505912 (494K) [application/octet-stream]
Saving to: ‘PER-ORG-relations-combined-group-2.tsv’

PER-ORG-relations-c 100%[===================>] 494.05K   567KB/s    in 0.9s    

2021-08-15 18:54:44 (567 KB/s) - ‘PER-ORG-relations-combined-group-2.tsv’ saved [505912/505912]

                                       label                                               text
0421f5ce-eab2-3eb4-9bec-a4067cd3c870  reject  Kuolleita 1676 – Pietro Francesco Caletti - Br...
17c04447-9a9f-37aa-9eba-73b368f33895  reject  <ORG>Terve . fi</ORG> - sivusto 29 . 10 . 2014...
2ac2440b-9cfe-3537-a81d-f8ff139f3ff5  accept  22 . syyskuuta 1937 <ORG>NKVD : n</ORG> agentt...
d147dc04-7271-3fec-bad8-611bb47a6c89  accept  H

In [ ]:
# jaetaan aineisto train- ja validation-osiin

x_train, x_valid, y_train, y_valid = train_test_split(data["text"], data["label"], test_size=0.2)
print("\nx_train:\n")
print(x_train.head())
print(x_train.shape)
print(y_train.head())

print("\nx_valid:\n")
print(x_valid.head())
print(x_valid.shape)
print(y_valid.head())


x_train:

15f58e4f-34a6-348c-97ee-cb9581b3c2fe    Hetken mietittyään <PERSON>Gillespie</PERSON> ...
16314214-dbb1-31c1-8cf9-349a22c060e8    Kesäkuussa 2012 <ORG>korkein oikeus</ORG> kats...
e295b08b-9a24-3813-8f5c-9eb84aa89e77    Vuodesta 1940 <PERSON>Alder</PERSON> työskente...
ae45ffae-b7a0-39b7-9477-d4944969546e    <ORG>Helsingin Sanomien</ORG> ulkomaantoimitta...
38d0fe55-087a-3b88-bd75-ea8fd33b00f4    <PERSON>Braunschweigin</PERSON> manifestissa 2...
Name: text, dtype: object
(1520,)
15f58e4f-34a6-348c-97ee-cb9581b3c2fe    reject
16314214-dbb1-31c1-8cf9-349a22c060e8    reject
e295b08b-9a24-3813-8f5c-9eb84aa89e77    accept
ae45ffae-b7a0-39b7-9477-d4944969546e    accept
38d0fe55-087a-3b88-bd75-ea8fd33b00f4    reject
Name: label, dtype: object

x_valid:

93095a66-7dc9-33b9-91c0-5a2faad89f0f    <PERSON>Kalluria</PERSON> ei koskaan varattu <...
e36c6e68-17a7-3a64-b4ee-f25e74bab093    <PERSON>Kudžijev</PERSON> kuului Edvard Gyllin...
02b828f5-79ec-3a7d-b0b7-b475d21b0ae5    Kuva : <PERS

In [ ]:
# setting model, dataset and hyperparameters

MODEL_NAME = 'TurkuNLP/bert-base-finnish-cased-v1'

LEARNING_RATE=1e-5
BATCH_SIZE=64
TRAIN_EPOCHS=1

# Magic value that comes from pytorch's default ignore_index
DUMMY_LABEL_ID = -100

In [ ]:
dataset = x_train
num_labels = len(set(dataset))
num_labels

1520

In [ ]:
print(dataset)
print(f'number of distinct labels: {num_labels}')

15f58e4f-34a6-348c-97ee-cb9581b3c2fe    Hetken mietittyään <PERSON>Gillespie</PERSON> ...
16314214-dbb1-31c1-8cf9-349a22c060e8    Kesäkuussa 2012 <ORG>korkein oikeus</ORG> kats...
e295b08b-9a24-3813-8f5c-9eb84aa89e77    Vuodesta 1940 <PERSON>Alder</PERSON> työskente...
ae45ffae-b7a0-39b7-9477-d4944969546e    <ORG>Helsingin Sanomien</ORG> ulkomaantoimitta...
38d0fe55-087a-3b88-bd75-ea8fd33b00f4    <PERSON>Braunschweigin</PERSON> manifestissa 2...
                                                              ...                        
d00cae43-ffaf-3936-b182-16bde6c9512f    Aiheesta muualla <PERSON>Matti Grönroosin</PER...
f3c9b207-dff6-39a7-a1a3-9e6d7eb50fca    Vuonna 1738 he muuttivat jälleen , tällä kerta...
7b07ca2f-d5f5-3459-858b-795b3c92c299    Kaudeksi 2010 – 2011 <PERSON>Hyvärinen</PERSON...
7006e716-995d-35db-a142-1cc8edbda691    Hän oli hallituksen ainoa kokoomuslainen ja <P...
baa9ff7d-dc4e-3018-8d85-6490e5140bdd    <PERSON>Iommin</PERSON> ainoaksi taltioiduksi ...
Name: text

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/816k [00:00<?, ?B/s]

In [ ]:
list(x_train)

['Hetken mietittyään <PERSON>Gillespie</PERSON> päätti teettää <ORG>Martin Companylla</ORG> vastaavanlaisen torven .',
 'Kesäkuussa 2012 <ORG>korkein oikeus</ORG> katsoi <PERSON>Jussi Halla - ahon</PERSON> syyllistyneen uskonrauhan rikkomiseen ja kiihottamiseen kansanryhmää vastaan ja kovensi hänen hovioikeudessa saamansa 40 päiväsakon tuomion 60 päiväsakoksi .',
 'Vuodesta 1940 <PERSON>Alder</PERSON> työskenteli soveltavan kemian ja kemianteknologian professorina <ORG>Kölnin yliopistossa</ORG> ja sittemmin hän oli yliopiston kemiallisen instituutin johtaja .',
 '<ORG>Helsingin Sanomien</ORG> ulkomaantoimittaja <PERSON>Heikki Aittokoski</PERSON> haastatteli lavalla kahta alkuperältään somalialaista kirjailijaa , jotka molemmat ovat kirjoittaneet Somalian historiaan sijoittuvan romaanin .',
 '<PERSON>Braunschweigin</PERSON> manifestissa 25 . heinäkuuta keisarillinen ja <ORG>Preussin armeija</ORG> uhkasivat kansaa pikaisella kostolla jos se vastustaisi kuningasvallan palauttamista tai ar

In [ ]:
def encode_dataset(texts, labels):
    tokenized = tokenizer(list(texts), is_split_into_words=False, truncation=True, max_length=150)
    labels_with_separators = []
    prev_word_idx = None
    for word_idx in tokenized.word_ids():
        if word_idx is None or word_idx == prev_word_idx:
            # Special token (e.g. [SEP]) or part of the previous word
            labels_with_separators.append(DUMMY_LABEL_ID)
        else:
            # Word start
            labels_with_separators.append(labels[word_idx])
        prev_word_idx = word_idx
    assert len(labels_with_separators) == len(tokenized.tokens())
    return (tokenized, labels_with_separators)


encoded_train_data, encoded_train_labels = encode_dataset(x_train, y_train)
encoded_validation_data, encoded_validation_labels = encode_dataset(x_valid, y_valid)

In [ ]:
set(encoded_train_labels)

{-100, 'accept', 'reject'}

In [ ]:
import torch

# Convert labels to numbers for torch.tensor
label_map = {
    '-100': -100,
    'accept': 0,
    'reject': 1
}

class TdaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        try:
            item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
            item['labels'] = label_map[str(self.labels[idx])]
            return item
        except Exception as e:
            print("Fail", idx) # Debug
            raise e

    def __len__(self):
        return len(self.labels)

train_dataset = TdaDataset(encoded_train_data, encoded_train_labels)
val_dataset = TdaDataset(encoded_validation_data, encoded_validation_labels)

In [ ]:
# Debug
{key: torch.tensor(val[29]) for key, val in val_dataset.encodings.items()}
torch.tensor(label_map[train_dataset.labels[29]])

tensor(1)

TODO: Ylläolevan ajaminen (ensimmäistä kertaa), tulostaa virheen

"Token indices sequence length is longer than the specified maximum sequence length for this model (148796 > 512). Running this sequence through the model will result in indexing errors"

Mistä johtuu? 

Edit: truncation=True ehkä korjaa?

In [ ]:
lengths = [len(x) for x in x_train]
print(max(lengths), len(x_train))

lengths = [len(x) for x in y_train]
print(max(lengths), len(y_train))


3439 1520
6 1520


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at TurkuNLP/bert-base-finnish-cased-v1 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the m

In [ ]:
train_args = TrainingArguments(
    'output_dir',    # output directory for checkpoints and predictions
    save_strategy='no',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    num_train_epochs=TRAIN_EPOCHS,
)

In [ ]:
def compute_accuracy(pred):
    y_pred = pred.predictions.argmax(axis=1)
    y_true = pred.label_ids
    return { 'accuracy': sum(y_pred == y_true) / len(y_true) }

In [ ]:
data_collator = DataCollatorForTokenClassification(
    tokenizer,
    label_pad_token_id=DUMMY_LABEL_ID
)

trainer = Trainer(
      model,
      train_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      # data_collator=data_collator,
      compute_metrics=compute_accuracy
)
trainer.train()

***** Running training *****
  Num examples = 41
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1


RuntimeError: ignored

***** Running training *****
  Num examples = 4
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1


IndexError: ignored